In [1]:
import gradio as gr
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI

from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.vectorstores import Chroma

In [3]:
api_key = os.getenv("OPENAI_API_KEY") 

In [4]:
chat = ChatOpenAI()
embedding_function = HuggingFaceEmbeddings(model_name = "BAAI/bge-large-en-v1.5",model_kwargs={'device': 'cpu'},encode_kwargs={"normalize_embeddings": True})

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6284\966817116.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6284\966817116.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name = "BAAI/bge-large-en-v1.5",model_kwargs={'device': 'cpu'},en

In [5]:

from langchain.text_splitter import CharacterTextSplitter

In [6]:
from langchain_community.vectorstores import FAISS

In [24]:
def add_docs(path):
    loader = PyPDFLoader(file_path=path)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    docs = loader.load_and_split(text_splitter=text_splitter)
    
                                                                                                
    db = FAISS.from_documents(documents=docs, embedding=embedding_function)
    return db






In [19]:
'''def answer_query(message, chat_history):
    base_compressor = LLMChainExtractor.from_llm(chat)
    base_retriever = db.as_retriever()
    mq_retriever = MultiQueryRetriever.from_llm(retriever = base_retriever, llm=chat)
    compression_retriever = ContextualCompressionRetriever(base_compressor=base_compressor, base_retriever=mq_retriever)

    #matched_docs = compression_retriever.get_relevant_documents(query = message)---->old version
    
    
    matched_docs = compression_retriever.invoke(message)

    context = ""

    for doc in matched_docs:
        page_content = doc.page_content
        context+=page_content
        context += "\n\n"
    template = """
    Answer the following question only by using the context given below in the triple backticks, do not use any other information to answer the question.
    If you can't answer the given question with the given context, you can return an emtpy string ('')
    Context: ```{context}```
    ----------------------------
    Question: {query}
    ----------------------------
    Answer: """
    
    human_message_prompt = HumanMessagePromptTemplate.from_template(template=template)
    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
    prompt = chat_prompt.format_prompt(query = message, context = context)
    #response = chat(messages=prompt.to_messages()).content---->old method
    
    
    #Updated method: invoke() instead of chat(messages=...)
    response = chat.invoke(prompt.to_messages()).content

    chat_history.append((message,response))
    return "", chat_history'''

'def answer_query(message, chat_history):\n    base_compressor = LLMChainExtractor.from_llm(chat)\n    base_retriever = db.as_retriever()\n    mq_retriever = MultiQueryRetriever.from_llm(retriever = base_retriever, llm=chat)\n    compression_retriever = ContextualCompressionRetriever(base_compressor=base_compressor, base_retriever=mq_retriever)\n\n    #matched_docs = compression_retriever.get_relevant_documents(query = message)---->old version\n    \n    \n    matched_docs = compression_retriever.invoke(message)\n\n    context = ""\n\n    for doc in matched_docs:\n        page_content = doc.page_content\n        context+=page_content\n        context += "\n\n"\n    template = """\n    Answer the following question only by using the context given below in the triple backticks, do not use any other information to answer the question.\n    If you can\'t answer the given question with the given context, you can return an emtpy string (\'\')\n    Context: ```{context}```\n    --------------

In [9]:
''''def answer_query(message, chat_history):
    # Create retrievers and compression retriever
    base_compressor = LLMChainExtractor.from_llm(chat)
    base_retriever = db.as_retriever()
    mq_retriever = MultiQueryRetriever.from_llm(retriever=base_retriever, llm=chat)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=base_compressor, base_retriever=mq_retriever
    )

    # Retrieve relevant documents
    matched_docs = compression_retriever.invoke(message)
    context = "\n\n".join(doc.page_content for doc in matched_docs)  # Simplified concatenation

    # Define the prompt
    template = """Answer using only the given context. If the context is insufficient, return an empty string ('').
    Context: ```{context}```
    ----------------------------
    Question: {query}
    ----------------------------
    Answer: """

    # Format the prompt and get response
    chat_prompt = ChatPromptTemplate.from_template(template)
    prompt = chat_prompt.format_prompt(query=message, context=context)
    response = chat.invoke(prompt.to_messages()).content  # Updated method

    # Append to chat history and return response
    chat_history.append((message, response))
    return response, chat_history'''




In [34]:
def answer_query(message, chat_history):
    
    base_compressor = LLMChainExtractor.from_llm(chat)
    base_retriever = db.as_retriever()
    mq_retriever = MultiQueryRetriever.from_llm(retriever=base_retriever, llm=chat)
    compression_retriever = ContextualCompressionRetriever(base_compressor=base_compressor, base_retriever=mq_retriever)

    matched_docs = compression_retriever.invoke(message)

    context = "\n\n".join(doc.page_content for doc in matched_docs)

    template = """
    Answer the following question only by using the context given below in the triple backticks, do not use any other information to answer the question.
    If you can't answer the given question with the given context, return an empty string ('').
    Context: ```{context}```
    ----------------------------
    Question: {query}
    ----------------------------
    Answer: """
    
    human_message_prompt = HumanMessagePromptTemplate.from_template(template=template)
    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
    prompt = chat_prompt.format_prompt(query = message, context = context)
    response = chat.invoke(prompt.to_messages()).content

    # Fix: Store messages as dictionaries
    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": response})

    return chat_history  # Return updated chat history

In [ ]:
# Load and process the document first
db = add_docs("Atomic habits.pdf")  # Ensure the PDF exists and is correct

# Then use db in answer_query()
message = "What are some atomic habits we should follow?"

output= answer_query(message, chat_history, db)

print(output)



In [12]:

''''import gradio as gr

with gr.Blocks() as demo:
    gr.HTML("<h1 align='center'>Smart Assistant</h1>")

    with gr.Row():
        upload_files = gr.File(label="Upload a PDF", file_types=[".pdf"], file_count="single")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Enter your question here")

    # Ensure document is uploaded before answering questions
    upload_files.upload(fn=add_docs, inputs=upload_files, outputs=None)

    # Submitting a question should process it and update chatbot
    msg.submit(fn=answer_query, inputs=[msg, chatbot], outputs=[chatbot])

# Launch UI
demo.launch()
"
chat_history = []  # Initialize an empty chat history

output, chat_history = answer_query(message, chat_history)

# Print the response
print(output)'''



SyntaxError: unterminated string literal (detected at line 1) (2164883569.py, line 1)

In [29]:
import gradio as gr

with gr.Blocks() as demo:
    gr.HTML("<h1 align='center'>DocuGenie By Saniya</h1>")

    with gr.Row():
        upload_files = gr.File(label="Upload a PDF", file_types=[".pdf"], file_count="single")

    chatbot = gr.Chatbot(type="messages")  #Fix: Use 'messages' type
    msg = gr.Textbox(label="Enter your question here")

    upload_files.upload(fn=add_docs, inputs=upload_files, outputs=None)
    msg.submit(fn=answer_query, inputs=[msg, chatbot], outputs=chatbot)  # ✅ Fix: No need to pass msg back

demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [35]:
import gradio as gr

with gr.Blocks() as demo:
    gr.HTML("<h1 align='center'>DocuGenie By Saniya 🧞‍♂️</h1>")

    with gr.Row():
        upload_files = gr.File(label="Upload a PDF📄", file_types=[".pdf"], file_count="single")

    chatbot = gr.Chatbot(label="💡 AI Assistant", type="messages")  #Fix: Use 'messages' type
    msg = gr.Textbox(label="Enter your question here")

    upload_files.upload(fn=add_docs, inputs=upload_files, outputs=None)
    msg.submit(fn=answer_query, inputs=[msg, chatbot], outputs=chatbot)  # ✅ Fix: No need to pass msg back

demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


D:\Users\Lenovo\anaconda3\Lib\site-packages\gradio\blocks.py:1821: UserWarning: A function (add_docs) returned too many output values (needed: 0, returned: 1). Ignoring extra values.
    Output components:
        []
    Output values returned:
  warnings.warn(
D:\Users\Lenovo\anaconda3\Lib\site-packages\gradio\blocks.py:1821: UserWarning: A function (add_docs) returned too many output values (needed: 0, returned: 1). Ignoring extra values.
    Output components:
        []
    Output values returned:
  warnings.warn(


D:\Users\Lenovo\anaconda3\Lib\site-packages\gradio\blocks.py:1821: UserWarning: A function (add_docs) returned too many output values (needed: 0, returned: 1). Ignoring extra values.
    Output components:
        []
    Output values returned:
  warnings.warn(
D:\Users\Lenovo\anaconda3\Lib\site-packages\gradio\blocks.py:1821: UserWarning: A function (add_docs) returned too many output values (needed: 0, returned: 1). Ignoring extra values.
    Output components:
        []
    Output values returned:
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
